In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import re
import os

In [ ]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

In [ ]:
try:
  import google.colab
  IN_COLAB = True
except:
  IN_COLAB = False

In [ ]:
import os
if IN_COLAB:
    print("running in google colab")
    from google.colab import drive
    drive.mount("/content/drive")
    os.chdir("/content/drive/My Drive/seminar_dl_workspace")
    print("switched workspace:", os.getcwd())
    
from create_visualizations import get_filenames_in_dir

In [ ]:
from IPython.display import SVG, display
def show_svg(filename):
    display(SVG(filename=filename))

boilerplate done...

In [ ]:
batch_sizes = [4, 8, 16, 32, 64, 128, 256, 512, 1024, 2048]

def get_batch_size_from_experiment_name(name):
    matches = re.search(r"([0-9]*)\.csv$", name)
    batch_size = int(matches.group(1))
    return batch_size

def get_latest_experiment_df(csv_path):
    experiment_stats = pd.read_csv(csv_path)
    experiment_stats_grouped = experiment_stats.groupby("experiment_id")
    latest_training_session = experiment_stats_grouped.get_group(experiment_stats.experiment_id.max())
    if len(experiment_stats) < 20:
        print(f"WARNING: file {csv_path} has only {len(experiment_stats)} epochs but should have 20")
    elif len(experiment_stats) > 20 and len(experiment_stats) % 20 == 0:
        print(f"INFO: file {csv_path} has {experiment_stats_grouped.ngroups} training sessions. Using latest training session with {len(latest_training_session)} epochs and experiment_id {latest_training_session.iloc[0]['experiment_id']}")
    return latest_training_session

In [ ]:
amount = len(batch_sizes)
results_test_acc = np.zeros(amount)
results_train_acc = np.zeros(amount)
results_test_avg_loss = np.zeros(amount)
results_train_avg_loss = np.zeros(amount)

experiments_stats_filenames = get_filenames_in_dir("experiments_stats", lambda x: "Adagrad_batch_size_" in x)
print(f"found {len(experiments_stats_filenames)} experiments for Adagrad_batch_size")
for filename in experiments_stats_filenames:
    batch_size = get_batch_size_from_experiment_name(filename)
    i = batch_sizes.index(batch_size)
    df = get_latest_experiment_df(os.path.join("experiments_stats" , filename))
    results_test_acc[i] = df["test_acc"].max()
    results_train_acc[i] = df["train_acc"].max()
    results_test_avg_loss[i] = df["test_avg_loss"].min()
    results_train_avg_loss[i] = df["train_avg_loss"].min()

max_train_acc = results_train_acc.max()
max_test_acc = results_test_acc.max()
min_train_loss = results_train_avg_loss.min()
min_test_loss = results_test_avg_loss.min()
print(f"max train accuracy: batch_size={batch_sizes[np.where(results_train_acc == max_train_acc)[0][0]]} --> {max_train_acc}")
print(f"max test accuracy: batch_size={batch_sizes[np.where(results_test_acc == max_test_acc)[0][0]]} --> {max_test_acc}")
print(f"min train avg loss: batch_size={batch_sizes[np.where(results_train_avg_loss == min_train_loss)[0][0]]} --> {min_train_loss}")
print(f"min test avg loss: batch_size={batch_sizes[np.where(results_test_avg_loss == min_test_loss)[0][0]]} --> {min_test_loss}")

In [ ]:
def draw_subplot_bar(ax, data, title, ylabel="", ylim=None):
    ax.bar(list(map(lambda x: str(x), batch_sizes)), data)
    ax.set_title(title)
    ax.set_ylabel(ylabel)
    ax.set_xlabel("batch size")
    bar_label_y_padding = 0
    bar_label_y_pos = 0
    if ylim is not None:
        ax.set_ylim(ylim)
        bar_label_y_padding = (ylim[1] - ylim[0]) * 0.05
        bar_label_y_pos = ylim[0]
    for i, v in enumerate(data):
        ax.text(i - 0.38, bar_label_y_pos + bar_label_y_padding, round(v, 3), fontsize=18, color="black")

Plot avg losses and accuracies for different batch sizes

In [ ]:
fig, axs = plt.subplots(2, 2, figsize=(28, 15), gridspec_kw=dict(hspace=0.35))
fig.suptitle("Adagrad for different batch sizes after 20 epochs")

draw_subplot_bar(axs[0, 0], results_train_acc, "Train Accuracy for different batch sizes", ylabel="accuracy", ylim=(0.7, 1.0))
draw_subplot_bar(axs[0, 1], results_test_acc, "Test Accuracy for different batch sizes", ylabel="accuracy", ylim=(0.5, 0.8))
draw_subplot_bar(axs[1, 0], results_train_avg_loss, "Train avg loss for different batch sizes", ylabel="loss", ylim=(0.0, 0.7))
draw_subplot_bar(axs[1, 1], results_test_avg_loss, "Test avg loss for different batch sizes", ylabel="loss", ylim=(0.0, 0.9))

plt.savefig("visualizations/adagrad_batch_sizes.png")